In [1]:
import pandas as pd
pd.options.display.max_rows = 100
orderlines = pd.read_csv('orderlines.csv')

In [2]:
#Removed 36169 rows with prices with two dots
#Should we delete the SKUs we have deleted in the table products because of weired prices?
#Check for number of orders

In [3]:
orderlines

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01


In [4]:
orderlines_df = orderlines.copy()

In [5]:
#chage data type to date
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

In [6]:
#orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [7]:
#How many numbers contain two dots
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

unit_price
False    257814
True      36169
Name: count, dtype: int64

In [8]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


In [9]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [10]:
orderlines_df.shape[0]

216250

In [11]:
#orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [12]:
orderlines_df.loc[:, "unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [13]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   id_order          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  object        
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 13.2+ MB


In [14]:
orderlines_df.loc[orderlines_df['unit_price'] == 0]

,id,id_order,product_id,product_quantity,sku,unit_price,date
53515,1227566,345934,0,1,KIN0153-2,0.0,2017-04-13 13:47:21
53530,1227590,345957,0,1,WDT0347,0.0,2017-04-13 14:44:05
57811,1234943,349487,0,2,LIBRO,0.0,2017-04-25 10:19:14
58425,1236042,349974,0,1,LIBRO,0.0,2017-04-26 13:48:27
58529,1236215,350062,0,1,LIBRO,0.0,2017-04-26 17:06:32
...,...,...,...,...,...,...,...
291880,1646431,525595,0,1,SYN0150,0.0,2018-03-11 20:50:46
292216,1647014,525883,0,1,APP0699,0.0,2018-03-12 12:26:57
292388,1647334,526025,0,1,WAC0251,0.0,2018-03-12 16:31:39
292905,1648288,526430,0,1,OWC0110,0.0,2018-03-13 13:27:05


In [15]:
orderlines_df.loc[orderlines_df['unit_price'] < 0]

,id,id_order,product_id,product_quantity,sku,unit_price,date
77008,1268645,365886,0,1,APP1465,-119.0,2017-06-15 12:48:54


In [16]:
orderlines_df['id_order'].nunique()

170213

In [17]:
orderlines_df['id_order'].count()

216250

In [18]:
orderlines_df['sku'].nunique()

6798

In [19]:
orderlines_df['unit_price'].max()

999.99

In [20]:
#Categorizing in price groups
bins = [0, 50, 100, 200, 500, 750, 1000, 2500, 5000, 7500, 10000, 12500, 16000]  
labels = ['-50', '-100', '-200', '-500', '-750', '-1000', '-2500', '-5000', '-7500', '-10000', '-12500', '-16000'] 

orderlines_df['price_group'] = pd.cut(orderlines_df['unit_price'], bins=bins, labels=labels, right=False)

In [21]:
orderlines_df.groupby('price_group')['sku'].count()

price_group
-50       86682
-100      45725
-200      32998
-500      30971
-750       9423
-1000     10450
-2500         0
-5000         0
-7500         0
-10000        0
-12500        0
-16000        0
Name: sku, dtype: int64

In [22]:
#Calculating average price: Revenue/number of products sold - or-
#Sum of Unit_Prices/Sum of SKUs

In [23]:
round(orderlines_df['unit_price'].sum())

35602570

In [24]:
orderlines_df['sku'].count()

216250

In [25]:
# 35.602.570 / 216.250 = 165 Euro

In [26]:
orderlines_cl = orderlines_df

In [27]:
orderlines_cl.to_csv("orderlines_cl.csv", index=False)